In [ ]:
# Imports

import car
from auto import console
import time

import numpy as np
import cv2
import os

Instantiated a global camera object!
Captured 1 frame.


TypeError: write() argument must be str, not numpy.ndarray

In [ ]:
# Variables
cv2.setNumThreads(0)

blue = [75, 110, 240]
diff = 20

crop_height = 70
crop_left = 60
crop_right = 60

# Be aware that opencv loads image in BGR format,
# that's why the color values have been adjusted here:
boundaries = [([blue[2], blue[1]-diff, blue[0]-diff],
        [blue[2]+diff, blue[1]+diff, blue[0]+diff])]

In [ ]:
# Functions

def crop(img):
    """
    Camera captures 240 pixels of height and 320 of width
    Shape is (240, 320, 3)
    """
    return img[-crop_height:, crop_left:-crop_right]


def scale(img, scalePercent):
    # Calculate the new dimensions
    width = int(img.shape[1] * scalePercent)
    height = int(img.shape[0] * scalePercent)
    newSize = (width, height)

    # Resize the image:
    img = cv2.resize(img, newSize, None, None, None, cv2.INTER_AREA)
    return img


def percent_blue(img):
    for (lower, upper) in boundaries:

        # You get the lower and upper part of the interval:
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)

        # cv2.inRange is used to binarize (i.e., render in white/black) an image
        # All the pixels that fall inside your interval [lower, uipper] will be white
        # All the pixels that do not fall inside this interval will
        # be rendered in black, for all three channels:
        mask = cv2.inRange(img, lower, upper)

        # Now, you AND the mask and the input image
        # All the pixels that are white in the mask will
        # survive the AND operation, all the black pixels
        # will remain black
        output = cv2.bitwise_and(img, img, mask=mask)

        # You can use the mask to count the number of white pixels.
        # Remember that the white pixels in the mask are those that
        # fall in your defined range, that is, every white pixel corresponds
        # to a blue pixel. Divide by the image size and you got the
        # percentage of blue pixels in the original image:
        ratio_blue = cv2.countNonZero(mask)/(img.size/3)

        # This is the color percent calculation, considering the resize I did earlier.
        colorPercent = (ratio_blue * 100) / scalePercent

        return colorPercent

In [ ]:
# Main logic

frame = car.capture()
img = crop(frame)

percentage = percent_blue(img)
console.print(percentage)